# Leetcode 300-399
See https://leetcode.com/problems/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import heapq
from typing import *

In [ ]:
# utility functions and version check

def run_sample(obj, sample, funcname):
    lines=[s for s in sample.splitlines() if len(s)>0 ]
    data=[json.loads(s) for s in lines]
    func=getattr(obj, funcname)
    res=func(*data[:-1])
    if res!=data[-1]:
        raise ValueError(f'expected {data[-1]}, actual {res}')

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # leetcode nnn)

In [ ]:
# leetcode 300
# https://leetcode.com/problems/longest-increasing-subsequence/
# tags: dynamic

sample1='''
[10,9,2,5,3,7,101,18]
4
'''

sample2='''
[0,1,0,3,2,3]
4
'''

sample3='''
[7,7,7,7,7,7,7]
1
'''

sample4='''
[-1]
1
'''

class Solution:
    # leetcode 300, idea: each running subsequence can be defined by its length up to then
    # and last number, so a tuple of (lastnum, len), there's max 2500 of them,
    # but note that if you have subseq (a, b) already then (a+1, b-1) doesn't make sense because
    # when they're the same length or shorter you would pick the lowest lastnum,
    # for each new num you have a choice of appending or skipping,
    # so you add max one new running sequence (note all the ones you append
    # can be folded into one longest), so every turn you want to know: list of all running subsequences,
    # for a certain number what's the longest subsequence below that number, let's do a naive quadratic 
    # implementation first (nice that this one is already fast enough, but you can keep them sorted,
    # both lastnum and len should be strictly increasing (decreasing length only makes sense if you
    # decrease lastnum also), then could use binary search to find maxtup quickly and also clean up 
    # existing subseqs that are outclassed by the new entry (or vice versa)
    def lengthOfLIS(self, nums: List[int]) -> int:
        subseqs=set()
        for n in nums:
            maxtup=None # best subseq to extend with n
            for tup in subseqs:
                if tup[0]<n and (maxtup is None or tup[1]>maxtup[1]):
                    maxtup=tup
            if maxtup is None:
                subseqs.add( (n, 1) )
            else:
                if maxtup[0]==n-1:
                    subseqs.remove(maxtup)
                subseqs.add( (n, maxtup[1]+1) )
        res=sorted(subseqs, key=lambda tup: -tup[1])
        return res[0][1]

run_sample(Solution(), sample1, 'lengthOfLIS')
run_sample(Solution(), sample2, 'lengthOfLIS')
run_sample(Solution(), sample3, 'lengthOfLIS')
run_sample(Solution(), sample4, 'lengthOfLIS')

In [ ]:
# leetcode 322
# https://leetcode.com/problems/coin-change/
# tags: dynamic

sample1='''
[1,2,5]
11
3
'''

sample2='''
[2]
3
-1
'''

sample3='''
[1]
0
0
'''

class Solution:
    # leetcode 322, idea: max 12 coins, so you can imagine a position as a vector of 12 numbers,
    # moving from position to position by adding one coin in the vector, could be a candidate for
    # DFS where you could start by adding highest possible value, backtrack when needed and prune 
    # based on an already succesful minimum total number of coins,
    # BFS should also be possible; as the amount is at most 10k there are also no more than 10k
    # different positions needed (could map each intermediate amount to the vector with the lowest amount 
    # of coins to get there)
    # A linear scan through intermediate amounts seems also possible; amount 0 is mapped to a vector of all zeroes,
    # for each other amount go through all coins, go the coin amount back, take the vector with least coins
    # there and add one, now from those 12 select the option with least coins
    def coinChange(self, coins: List[int], amount: int) -> int:
        bestvectors={} # maps amount to best list of numbers of coins (same positions as coins) plus total amount
        zero=[]
        for _ in coins:
            zero.append(0)
        zero.append(0) # total count is 0
        bestvectors[0]=zero
        for amt in range(1, amount+1):
            bv=None # best vector so far
            for i, coin in enumerate(coins):
                cand=bestvectors.get(amt-coin)
                if cand is None:
                    continue
                if bv is None or cand[-1]+1<bv[-1]:
                    bv=list(cand)
                    bv[i]+=1
                    bv[-1]+=1
            if bv is not None:
                bestvectors[amt]=bv
        bv=bestvectors.get(amount)
        return -1 if bv is None else bv[-1]

run_sample(Solution(), sample1, 'coinChange')
run_sample(Solution(), sample2, 'coinChange')
run_sample(Solution(), sample3, 'coinChange')

In [ ]:
# leetcode 326
# https://leetcode.com/problems/power-of-three/

sample1='''
27
true
'''

sample2='''
0
false
'''

sample3='''
-1
false
'''

sample4='''
18
false
'''

sample5='''
1
true
'''

class Solution:
    # leetcode 327
    def isPowerOfThree(self, n: int) -> bool:
        if n<=0:
            return False
        while n%3==0:
            n//=3
        return n==1

run_sample(Solution(), sample1, 'isPowerOfThree')
run_sample(Solution(), sample2, 'isPowerOfThree')
run_sample(Solution(), sample3, 'isPowerOfThree')
run_sample(Solution(), sample4, 'isPowerOfThree')
run_sample(Solution(), sample5, 'isPowerOfThree')

In [ ]:
# leetcode 328
# https://leetcode.com/problems/odd-even-linked-list/

sample1='''
[1,2,3,4,5]
[1,3,5,2,4]
'''

sample2='''
[2,1,3,5,6,4,7]
[2,3,6,7,1,5,4]
'''

sample3='''
[]
[]
'''

# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

class Solution:
    def oddEvenList2(self, l1: List[int]) -> List[int]:
        y=None
        for n in l1[::-1]:
            x=ListNode(n, y)
            y=x
        l1x=y
        rx=self.oddEvenList(l1x)
        res=[]
        while rx is not None:
            res.append(rx.val)
            rx=rx.next
        return res

    # leetcode 328, idea: maintain head and tail of odd list and head and tail of even list
    def oddEvenList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        head_odd=None
        tail_odd=None
        head_even=None
        tail_even=None
        node=head
        is_odd=True
        while node is not None:
            if is_odd:
                if head_odd is None:
                    head_odd=node
                    tail_odd=node
                else:
                    tail_odd.next=node
                    tail_odd=node
            else:
                if head_even is None:
                    head_even=node
                    tail_even=node
                else:
                    tail_even.next=node
                    tail_even=node
            node_tmp=node
            node=node.next
            node_tmp.next=None
            is_odd=not is_odd
        # now concatenate
        if head_odd is None:
            head_odd=head_even
        else:
            tail_odd.next=head_even
        return head_odd

run_sample(Solution(), sample1, 'oddEvenList2')
run_sample(Solution(), sample2, 'oddEvenList2')
run_sample(Solution(), sample3, 'oddEvenList2')

In [ ]:
# leetcode 334
# https://leetcode.com/problems/increasing-triplet-subsequence/

sample1='''
[1,2,3,4,5]
true
'''

sample2='''
[5,4,3,2,1]
false
'''

sample3='''
[2,1,5,0,4,6]
true
'''

sample4='''
[16]
false
'''

sample5='''
[20,100,10,12,5,13]
true
'''

class Solution:
    # leetcode 334, idea: try to build array of three matching elements going from start to finish
    # - if you have 0 or 1 element can just replace by a lower element
    # - if you have two elements already and find a higher you're done, if you find lower than second you can just 
    #   replace, if you find lower than first just start a second array
    # - so you only need three variables
    def increasingTriplet(self, nums: List[int]) -> bool:
        a=None # first array first elem
        b=None # first array second elem
        d=None # second array first elem
        for n in nums:
            if a is None:
                assert b is None
                a=n
            elif b is None:
                assert d is None
                if n<a:
                    a=n
                if n>a:
                    b=n
            elif d is None:
                if n>b:
                    return True
                if n>a:
                    b=n
                if n<a:
                    d=n
            else: # all three spots are full
                assert d<a
                if n>b:
                    return True
                if n<d:
                    d=n
                if d<n<b:
                    a=d
                    b=n
                    d=None
        return False

run_sample(Solution(), sample1, 'increasingTriplet')
run_sample(Solution(), sample2, 'increasingTriplet')
run_sample(Solution(), sample3, 'increasingTriplet')
run_sample(Solution(), sample4, 'increasingTriplet')
run_sample(Solution(), sample5, 'increasingTriplet')

In [ ]:
# leetcode 344
# https://leetcode.com/problems/reverse-string/

sample1='''
["h","e","l","l","o"]
["o","l","l","e","h"]
'''

sample2='''
["H","a","n","n","a","h"]
["h","a","n","n","a","H"]
'''

class Solution:
    def reverseString2(self, s: List[str]) -> List[str]:
        self.reverseString(s)
        return s

    # leetcode 344 
    def reverseString(self, s: List[str]) -> None:
        for i in range(len(s)//2):
            saved=s[i]
            newindex= -1-i
            s[i]=s[newindex]
            s[newindex]=saved        

run_sample(Solution(), sample1, 'reverseString2')
run_sample(Solution(), sample2, 'reverseString2')

In [ ]:
# leetcode 347
# https://leetcode.com/problems/top-k-frequent-elements/

sample1='''
[1,1,1,2,2,3]
2
[1,2]
'''

sample2='''
[1]
1
[1]
'''

class Solution:
    # leetcode 347, idea: just counter and sort should be fine?
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        cnt=collections.Counter(nums)
        toptups=sorted(cnt.items(), key=lambda tup: -tup[1])[:k]
        return [tup[0] for tup in toptups]

run_sample(Solution(), sample1, 'topKFrequent')
run_sample(Solution(), sample2, 'topKFrequent')

In [ ]:
# leetcode 350
# https://leetcode.com/problems/intersection-of-two-arrays-ii/

sample1='''
[1,2,2,1]
[2,2]
[2,2]
'''

sample2='''
[4,9,5]
[9,4,9,8,4]
[4,9]
'''

class Solution:
    # leetcode 350
    def intersect(self, nums1: List[int], nums2: List[int]) -> List[int]:
        c1=collections.Counter(nums1)
        c2=collections.Counter(nums2)
        c3={ k: min(v, c2[k]) for k,v in c1.items() if k in c2 }
        res=[[k]*v for k,v in c3.items()]
        res=list(itertools.chain(*res))
        return res

run_sample(Solution(), sample1, 'intersect')
run_sample(Solution(), sample2, 'intersect')

In [ ]:
# leetcode 380
# https://leetcode.com/problems/insert-delete-getrandom-o1/

import random

class RandomizedSet:
    # leetcode 380
    def __init__(self):
        self.data=set()

    def insert(self, val: int) -> bool:
        res=val not in self.data
        self.data.add(val)
        return res

    def remove(self, val: int) -> bool:
        if val in self.data:
            self.data.remove(val)
            return True
        else:
            return False

    def getRandom(self) -> int:
        assert len(self.data)>0
        res=random.sample(self.data, 1)
        return res[0]
        

In [ ]:
# leetcode 384
# https://leetcode.com/problems/shuffle-an-array/

import random

class Solution:
    # leetcode 384
    def __init__(self, nums: List[int]):
        self.backup=list(nums)        

    def reset(self) -> List[int]:
        return list(self.backup)        

    def shuffle(self) -> List[int]:
        return random.sample(self.backup, k=len(self.backup))


In [ ]:
# leetcode 387
# https://leetcode.com/problems/first-unique-character-in-a-string/

sample1='''
"leetcode"
0
'''

sample2='''
"loveleetcode"
2
'''

sample3='''
"aabb"
-1
'''

sample4='''
""
-1
'''

class Solution:
    # leetcode 387
    def firstUniqChar(self, s: str) -> int:
        cnt=collections.Counter(list(s))
        for i in range(len(s)):
            if cnt[s[i]]<2:
                return i
        return -1

run_sample(Solution(), sample1, 'firstUniqChar')
run_sample(Solution(), sample2, 'firstUniqChar')
run_sample(Solution(), sample3, 'firstUniqChar')
run_sample(Solution(), sample4, 'firstUniqChar')